In [3]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt

In [2]:
sales = pd.read_csv("data/sales_cleaned.csv")
logs = pd.read_csv("data/logs.csv")
tickets = pd.read_csv("data/dataset3.csv")


In [4]:
print(tickets.columns)

Index(['ticket_id', 'customer_id', 'created_at', 'channel', 'priority',
       'issue_category', 'ticket_text', 'sentiment', 'first_response_hours',
       'resolution_hours', 'resolved'],
      dtype='object')


In [5]:
print(tickets.dtypes)

ticket_id                 int64
customer_id              object
created_at               object
channel                  object
priority                 object
issue_category           object
ticket_text              object
sentiment               float64
first_response_hours    float64
resolution_hours        float64
resolved                  int64
dtype: object


In [3]:
tickets.head()

,ticket_id,customer_id,created_at,channel,priority,issue_category,ticket_text,sentiment,first_response_hours,resolution_hours,resolved
0,200001,C100000,2024-05-12T23:17,Chat,Medium,product_performance,Dashboard loads very slowly during peak hours.,0.44,0.11,14.88,1
1,200002,C100001,2024-06-28T18:27,Email,Low,product_performance,Dashboard loads very slowly during peak hours.,-0.50,8.12,9.37,1
2,200003,C100002,2024-11-25T16:38,Email,Medium,product_usability,The new dashboard is confusing for my staff.,-0.50,2.97,31.65,1
3,200004,C100003,2024-11-02T14:37,Phone,High,billing_admin,I need to change our billing email and can't f...,-0.11,0.10,12.49,1
4,200005,C100004,2024-10-17T22:27,Phone,Medium,product_performance,Inventory sync takes too long to complete.,0.20,5.19,35.67,1


In [11]:
tickets.isna().sum()

customer_id                 0
Q3_Status                   0
channel                     0
priority                    0
issue_category              0
ticket_text                 0
avg_sentiment               0
avg_first_response_hours    0
avg_resolution_hours        0
avg_resolved                0
ticket_count                0
dtype: int64

In [6]:
# create before and after Q3 time period
tickets["created_at"] = pd.to_datetime(tickets["created_at"])

month_number = tickets['created_at'].dt.month
is_q3_or_later = month_number >= 7

tickets['Q3_Status'] = np.where(
    is_q3_or_later,
    'During/After Q3',  # Value if True (Month is 7, 8, 9, 10, 11, or 12)
    'Before Q3'         # Value if False (Month is 1, 2, 3, 4, 5, or 6)
)

#### Analyze Issue text
* Unique ticket_text in each issue_category
* product_performance
    * 'Dashboard loads very slowly during peak hours.',
    * 'Inventory sync takes too long to complete.',
    * 'The reporting page keeps timing out.',
    * 'Mobile app crashes when I try to view inventory levels.'
* product_usability
    * 'The new dashboard is confusing for my staff.',
    * 'Bulk upload flow is not intuitive.',
    * 'It's hard to find where to update product SKUs.',
    * 'Search results are not clearly sorted.'
* billing_admin
    * 'I need to change our billing email and can't find the option.',
    * 'We were charged for extra users we don't have.',
    * 'Our discount was not applied to this renewal.',
    * 'VAT appears to be calculated incorrectly.'
* sales_expectation
    * 'Pricing discussed in the demo does not match our invoice.',
    * 'We were told onboarding would be fully managed, but it wasn't.',
    * 'Sales promised a custom report that doesn't exist.',
    * 'We expected multi-warehouse support out of the box.'
* From the look of percent distribution of issue_text within issue_category, it's pretty evenly distributioned. BUT maybe there will be a correlation with the is_churned once we aggregate it.

In [11]:
tickets["issue_category"].unique()

array(['product_performance', 'product_usability', 'billing_admin',
       'sales_expectation'], dtype=object)

In [12]:
tickets[tickets["issue_category"] == "sales_expectation"]["ticket_text"].unique()

array(['Pricing discussed in the demo does not match our invoice.',
       "We were told onboarding would be fully managed, but it wasn't.",
       "Sales promised a custom report that doesn't exist.",
       'We expected multi-warehouse support out of the box.'],
      dtype=object)

In [13]:
# check percent of issue_text in each issue_category
grouped = tickets.groupby(['issue_category', 'ticket_text']).size().reset_index(name='count')

# 2. Calculate the total for each Parent Category
# We use .transform('sum') to broadcast the parent's total to every child row
grouped['parent_total'] = grouped.groupby('issue_category')['count'].transform('sum')

# 3. Calculate the Percentage
grouped['percent_of_parent'] = (grouped['count'] / grouped['parent_total']) * 100

# Optional: Cleanup for readability
grouped['percent_of_parent'] = grouped['percent_of_parent'].round(2)

print(grouped)

         issue_category                                        ticket_text  \
0         billing_admin  I need to change our billing email and can't f...   
1         billing_admin      Our discount was not applied to this renewal.   
2         billing_admin          VAT appears to be calculated incorrectly.   
3         billing_admin     We were charged for extra users we don't have.   
4   product_performance     Dashboard loads very slowly during peak hours.   
5   product_performance         Inventory sync takes too long to complete.   
6   product_performance  Mobile app crashes when I try to view inventor...   
7   product_performance               The reporting page keeps timing out.   
8     product_usability                 Bulk upload flow is not intuitive.   
9     product_usability    It's hard to find where to update product SKUs.   
10    product_usability             Search results are not clearly sorted.   
11    product_usability       The new dashboard is confusing for

#### Aggregation to Customer Level
* Group by Key: cusomter_id, q3 status, channel, priority, issue_category, ticket_text
* sentiment - average value
* first_response_hours - average value   
* resolution_hours - average value    
* resolved - proportion                 

In [ ]:
grouped = tickets.groupby([
    "customer_id", 
    "Q3_Status", 
    'channel', 
    "priority", 
    "issue_category",
    "ticket_text"
])[["sentiment", "first_response_hours", "resolution_hours", "resolved"]].mean()

In [ ]:
grouped_tickets = tickets.groupby([
    "customer_id", 
    "Q3_Status", #2
    'channel', #4
    "priority", #3
    "issue_category", #4
    "ticket_text" #16
]).agg(
    # --- Mean Aggregations (renaming with 'avg_' prefix) ---
    avg_sentiment=('sentiment', 'mean'),
    avg_first_response_hours=('first_response_hours', 'mean'),
    avg_resolution_hours=('resolution_hours', 'mean'),
    avg_resolved=('resolved', 'mean'),
    
    # --- Count Aggregation (renaming the count column) ---
    ticket_count=('customer_id', 'count') 
)

# Reset the index to turn the grouping keys into standard columns
grouped_df = grouped_tickets.reset_index()

print(f'Size of Tickets: {len(tickets)}')
print(f'Size of Tickets after Group by Aggregate: {len(grouped_df)}')
grouped_df.head()

Size of Tickets: 6429
Size of Tickets after Group by Aggregate: 6394


,customer_id,Q3_Status,channel,priority,issue_category,ticket_text,avg_sentiment,avg_first_response_hours,avg_resolution_hours,avg_resolved,ticket_count
0,C100000,Before Q3,Chat,Medium,product_performance,Dashboard loads very slowly during peak hours.,0.44,0.11,14.88,1.0,1
1,C100001,Before Q3,Email,Low,product_performance,Dashboard loads very slowly during peak hours.,-0.50,8.12,9.37,1.0,1
2,C100002,During/After Q3,Email,Medium,product_usability,The new dashboard is confusing for my staff.,-0.50,2.97,31.65,1.0,1
3,C100003,During/After Q3,Phone,High,billing_admin,I need to change our billing email and can't f...,-0.11,0.10,12.49,1.0,1
4,C100004,During/After Q3,Phone,Medium,product_performance,Inventory sync takes too long to complete.,0.20,5.19,35.67,1.0,1


In [46]:
grouped_df[grouped_df["ticket_count"] > 1].sort_values(by=["customer_id"]).head()

,customer_id,Q3_Status,channel,priority,issue_category,ticket_text,avg_sentiment,avg_first_response_hours,avg_resolution_hours,avg_resolved,ticket_count
14,C100006,During/After Q3,Phone,Medium,product_performance,Inventory sync takes too long to complete.,-0.075,4.225,15.145,0.5,2
145,C100074,During/After Q3,Phone,Medium,billing_admin,VAT appears to be calculated incorrectly.,-0.410,3.270,7.600,1.0,2
302,C100134,During/After Q3,Phone,High,product_usability,Bulk upload flow is not intuitive.,0.085,3.865,10.305,1.0,2
440,C100198,During/After Q3,Email,Medium,product_usability,Bulk upload flow is not intuitive.,-0.295,0.325,9.870,1.0,2
681,C100290,During/After Q3,Chat,Medium,sales_expectation,Pricing discussed in the demo does not match o...,-0.075,1.965,13.330,0.5,2


In [45]:
grouped_df.head()

,customer_id,Q3_Status,channel,priority,issue_category,ticket_text,avg_sentiment,avg_first_response_hours,avg_resolution_hours,avg_resolved,ticket_count
0,C100000,Before Q3,Chat,Medium,product_performance,Dashboard loads very slowly during peak hours.,0.44,0.11,14.88,1.0,1
1,C100001,Before Q3,Email,Low,product_performance,Dashboard loads very slowly during peak hours.,-0.50,8.12,9.37,1.0,1
2,C100002,During/After Q3,Email,Medium,product_usability,The new dashboard is confusing for my staff.,-0.50,2.97,31.65,1.0,1
3,C100003,During/After Q3,Phone,High,billing_admin,I need to change our billing email and can't f...,-0.11,0.10,12.49,1.0,1
4,C100004,During/After Q3,Phone,Medium,product_performance,Inventory sync takes too long to complete.,0.20,5.19,35.67,1.0,1


In [47]:
grouped_df.to_csv("data/tickets_aggregated.csv")

In [4]:
df = pd.read_csv("data/tickets_aggregated.csv")
df.head()

,Unnamed: 0,customer_id,Q3_Status,channel,priority,issue_category,ticket_text,avg_sentiment,avg_first_response_hours,avg_resolution_hours,avg_resolved,ticket_count
0,0,C100000,Before Q3,Chat,Medium,product_performance,Dashboard loads very slowly during peak hours.,0.44,0.11,14.88,1.0,1
1,1,C100001,Before Q3,Email,Low,product_performance,Dashboard loads very slowly during peak hours.,-0.50,8.12,9.37,1.0,1
2,2,C100002,During/After Q3,Email,Medium,product_usability,The new dashboard is confusing for my staff.,-0.50,2.97,31.65,1.0,1
3,3,C100003,During/After Q3,Phone,High,billing_admin,I need to change our billing email and can't f...,-0.11,0.10,12.49,1.0,1
4,4,C100004,During/After Q3,Phone,Medium,product_performance,Inventory sync takes too long to complete.,0.20,5.19,35.67,1.0,1


#### Aggregate Tickets & Sales

In [5]:
sales = pd.read_csv("data/sales_cleaned.csv").drop(columns=["Unnamed: 0"], axis = 1)
tickets = pd.read_csv("data/tickets_aggregated.csv").drop(columns=["Unnamed: 0"], axis = 1)

In [7]:
sales.head()

,customer_id,company_name,country,region,is_eu,industry,company_size_bucket,annual_contract_value,product_tier,sales_segment,acquisition_channel,contract_start_date,contract_end_date,renewed_flag,discount_pct,initial_onboarding_score,is_churned
0,C100000,Company_0,Canada,North America,0,Healthcare,51-200,12999.25,Growth,MidMarket,Partner,2023-05-02,NaN,1,0.03,7.0,0
1,C100001,Company_1,UK,Europe,1,Wholesale,1-10,1799.72,Starter,SMB_Field,Inbound,2023-02-18,NaN,1,0.15,4.9,0
2,C100002,Company_2,US,North America,0,Manufacturing,1-10,1770.83,Starter,SMB_Field,Inbound,2023-11-12,2024-05-13,0,0.37,4.8,1
3,C100003,Company_3,France,Europe,1,Missing,1-10,1790.30,Starter,SMB_Field,Outbound,2023-08-06,NaN,1,0.10,8.0,0
4,C100004,Company_4,Netherlands,Europe,1,Professional Services,1-10,1552.00,Starter,SMB_Inside,Inbound,2023-09-09,2024-03-10,0,0.11,3.8,1


In [8]:
tickets.head()

,customer_id,Q3_Status,channel,priority,issue_category,ticket_text,avg_sentiment,avg_first_response_hours,avg_resolution_hours,avg_resolved,ticket_count
0,C100000,Before Q3,Chat,Medium,product_performance,Dashboard loads very slowly during peak hours.,0.44,0.11,14.88,1.0,1
1,C100001,Before Q3,Email,Low,product_performance,Dashboard loads very slowly during peak hours.,-0.50,8.12,9.37,1.0,1
2,C100002,During/After Q3,Email,Medium,product_usability,The new dashboard is confusing for my staff.,-0.50,2.97,31.65,1.0,1
3,C100003,During/After Q3,Phone,High,billing_admin,I need to change our billing email and can't f...,-0.11,0.10,12.49,1.0,1
4,C100004,During/After Q3,Phone,Medium,product_performance,Inventory sync takes too long to complete.,0.20,5.19,35.67,1.0,1


In [1]:
# tickets[tickets["ticket_count"] > 1]

In [8]:
sales.isna().sum()

customer_id                    0
company_name                   0
country                        0
region                         0
is_eu                          0
industry                       0
company_size_bucket            0
annual_contract_value          0
product_tier                   0
sales_segment                  0
acquisition_channel            0
contract_start_date            0
contract_end_date           1428
renewed_flag                   0
discount_pct                   0
initial_onboarding_score       0
is_churned                     0
dtype: int64

In [10]:
print(tickets.columns)
tickets.isna().sum()

Index(['customer_id', 'Q3_Status', 'channel', 'priority', 'issue_category',
       'ticket_text', 'avg_sentiment', 'avg_first_response_hours',
       'avg_resolution_hours', 'avg_resolved', 'ticket_count'],
      dtype='object')


customer_id                 0
Q3_Status                   0
channel                     0
priority                    0
issue_category              0
ticket_text                 0
avg_sentiment               0
avg_first_response_hours    0
avg_resolution_hours        0
avg_resolved                0
ticket_count                0
dtype: int64

In [6]:
joined = pd.merge(sales, tickets, on= "customer_id", how="left")
joined.head()

,customer_id,company_name,country,region,is_eu,industry,company_size_bucket,annual_contract_value,product_tier,sales_segment,...,Q3_Status,channel,priority,issue_category,ticket_text,avg_sentiment,avg_first_response_hours,avg_resolution_hours,avg_resolved,ticket_count
0,C100000,Company_0,Canada,North America,0,Healthcare,51-200,12999.25,Growth,MidMarket,...,Before Q3,Chat,Medium,product_performance,Dashboard loads very slowly during peak hours.,0.44,0.11,14.88,1.0,1.0
1,C100001,Company_1,UK,Europe,1,Wholesale,1-10,1799.72,Starter,SMB_Field,...,Before Q3,Email,Low,product_performance,Dashboard loads very slowly during peak hours.,-0.50,8.12,9.37,1.0,1.0
2,C100002,Company_2,US,North America,0,Manufacturing,1-10,1770.83,Starter,SMB_Field,...,During/After Q3,Email,Medium,product_usability,The new dashboard is confusing for my staff.,-0.50,2.97,31.65,1.0,1.0
3,C100003,Company_3,France,Europe,1,Missing,1-10,1790.30,Starter,SMB_Field,...,During/After Q3,Phone,High,billing_admin,I need to change our billing email and can't f...,-0.11,0.10,12.49,1.0,1.0
4,C100004,Company_4,Netherlands,Europe,1,Professional Services,1-10,1552.00,Starter,SMB_Inside,...,During/After Q3,Phone,Medium,product_performance,Inventory sync takes too long to complete.,0.20,5.19,35.67,1.0,1.0


In [7]:
joined.isna().sum()

customer_id                    0
company_name                   0
country                        0
region                         0
is_eu                          0
industry                       0
company_size_bucket            0
annual_contract_value          0
product_tier                   0
sales_segment                  0
acquisition_channel            0
contract_start_date            0
contract_end_date           3276
renewed_flag                   0
discount_pct                   0
initial_onboarding_score       0
is_churned                     0
Q3_Status                    579
channel                      579
priority                     579
issue_category               579
ticket_text                  579
avg_sentiment                579
avg_first_response_hours     579
avg_resolution_hours         579
avg_resolved                 579
ticket_count                 579
dtype: int64

In [15]:
cluster = pd.read_csv("data/customer_cluster_assignments.csv")
cluster.head()

,customer_id,kmeans_cluster,hierarchical_cluster,kprototype_cluster
0,C100000,2,3,1
1,C100001,1,4,0
2,C100002,0,2,2
3,C100003,1,3,1
4,C100004,1,4,0


In [16]:
joined = pd.merge(joined, cluster, on= "customer_id", how="left")
joined.head()

,customer_id,company_name,country,region,is_eu,industry,company_size_bucket,annual_contract_value,product_tier,sales_segment,...,issue_category,ticket_text,avg_sentiment,avg_first_response_hours,avg_resolution_hours,avg_resolved,ticket_count,kmeans_cluster,hierarchical_cluster,kprototype_cluster
0,C100000,Company_0,Canada,North America,0,Healthcare,51-200,12999.25,Growth,MidMarket,...,product_performance,Dashboard loads very slowly during peak hours.,0.44,0.11,14.88,1.0,1.0,2,3,1
1,C100001,Company_1,UK,Europe,1,Wholesale,1-10,1799.72,Starter,SMB_Field,...,product_performance,Dashboard loads very slowly during peak hours.,-0.50,8.12,9.37,1.0,1.0,1,4,0
2,C100002,Company_2,US,North America,0,Manufacturing,1-10,1770.83,Starter,SMB_Field,...,product_usability,The new dashboard is confusing for my staff.,-0.50,2.97,31.65,1.0,1.0,0,2,2
3,C100003,Company_3,France,Europe,1,Missing,1-10,1790.30,Starter,SMB_Field,...,billing_admin,I need to change our billing email and can't f...,-0.11,0.10,12.49,1.0,1.0,1,3,1
4,C100004,Company_4,Netherlands,Europe,1,Professional Services,1-10,1552.00,Starter,SMB_Inside,...,product_performance,Inventory sync takes too long to complete.,0.20,5.19,35.67,1.0,1.0,1,4,0


In [17]:
joined.to_csv("data/sale_cluster_tickets_joined.csv")

#### Get lost_revenue as target

In [21]:
joined = pd.read_csv("data/sale_cluster_tickets_joined.csv").drop(columns=["Unnamed: 0"], errors="ignore")
print(joined.columns)
joined.head(1)

Index(['customer_id', 'company_name', 'country', 'region', 'is_eu', 'industry',
       'company_size_bucket', 'annual_contract_value', 'product_tier',
       'sales_segment', 'acquisition_channel', 'contract_start_date',
       'contract_end_date', 'renewed_flag', 'discount_pct',
       'initial_onboarding_score', 'is_churned', 'Q3_Status', 'channel',
       'priority', 'issue_category', 'ticket_text', 'avg_sentiment',
       'avg_first_response_hours', 'avg_resolution_hours', 'avg_resolved',
       'ticket_count', 'kmeans_cluster', 'hierarchical_cluster',
       'kprototype_cluster'],
      dtype='object')


,customer_id,company_name,country,region,is_eu,industry,company_size_bucket,annual_contract_value,product_tier,sales_segment,...,issue_category,ticket_text,avg_sentiment,avg_first_response_hours,avg_resolution_hours,avg_resolved,ticket_count,kmeans_cluster,hierarchical_cluster,kprototype_cluster
0,C100000,Company_0,Canada,North America,0,Healthcare,51-200,12999.25,Growth,MidMarket,...,product_performance,Dashboard loads very slowly during peak hours.,0.44,0.11,14.88,1.0,1.0,2,3,1


In [22]:
# just use hierarchical clusters: drop kmeans_cluster & kprototype_cluster
# calculate lost revenue
joined = joined.drop(columns=["kmeans_cluster", "kprototype_cluster"], axis = 1)
joined.head()

,customer_id,company_name,country,region,is_eu,industry,company_size_bucket,annual_contract_value,product_tier,sales_segment,...,channel,priority,issue_category,ticket_text,avg_sentiment,avg_first_response_hours,avg_resolution_hours,avg_resolved,ticket_count,hierarchical_cluster
0,C100000,Company_0,Canada,North America,0,Healthcare,51-200,12999.25,Growth,MidMarket,...,Chat,Medium,product_performance,Dashboard loads very slowly during peak hours.,0.44,0.11,14.88,1.0,1.0,3
1,C100001,Company_1,UK,Europe,1,Wholesale,1-10,1799.72,Starter,SMB_Field,...,Email,Low,product_performance,Dashboard loads very slowly during peak hours.,-0.50,8.12,9.37,1.0,1.0,4
2,C100002,Company_2,US,North America,0,Manufacturing,1-10,1770.83,Starter,SMB_Field,...,Email,Medium,product_usability,The new dashboard is confusing for my staff.,-0.50,2.97,31.65,1.0,1.0,2
3,C100003,Company_3,France,Europe,1,Missing,1-10,1790.30,Starter,SMB_Field,...,Phone,High,billing_admin,I need to change our billing email and can't f...,-0.11,0.10,12.49,1.0,1.0,3
4,C100004,Company_4,Netherlands,Europe,1,Professional Services,1-10,1552.00,Starter,SMB_Inside,...,Phone,Medium,product_performance,Inventory sync takes too long to complete.,0.20,5.19,35.67,1.0,1.0,4


In [23]:
joined["lost_revenue"] = joined["is_churned"] * joined["annual_contract_value"]
joined.head()

,customer_id,company_name,country,region,is_eu,industry,company_size_bucket,annual_contract_value,product_tier,sales_segment,...,priority,issue_category,ticket_text,avg_sentiment,avg_first_response_hours,avg_resolution_hours,avg_resolved,ticket_count,hierarchical_cluster,lost_revenue
0,C100000,Company_0,Canada,North America,0,Healthcare,51-200,12999.25,Growth,MidMarket,...,Medium,product_performance,Dashboard loads very slowly during peak hours.,0.44,0.11,14.88,1.0,1.0,3,0.00
1,C100001,Company_1,UK,Europe,1,Wholesale,1-10,1799.72,Starter,SMB_Field,...,Low,product_performance,Dashboard loads very slowly during peak hours.,-0.50,8.12,9.37,1.0,1.0,4,0.00
2,C100002,Company_2,US,North America,0,Manufacturing,1-10,1770.83,Starter,SMB_Field,...,Medium,product_usability,The new dashboard is confusing for my staff.,-0.50,2.97,31.65,1.0,1.0,2,1770.83
3,C100003,Company_3,France,Europe,1,Missing,1-10,1790.30,Starter,SMB_Field,...,High,billing_admin,I need to change our billing email and can't f...,-0.11,0.10,12.49,1.0,1.0,3,0.00
4,C100004,Company_4,Netherlands,Europe,1,Professional Services,1-10,1552.00,Starter,SMB_Inside,...,Medium,product_performance,Inventory sync takes too long to complete.,0.20,5.19,35.67,1.0,1.0,4,1552.00


In [24]:
joined.to_csv("data/sale_cluster_tickets_regr.csv")

In [26]:
print(joined.columns)

Index(['customer_id', 'company_name', 'country', 'region', 'is_eu', 'industry',
       'company_size_bucket', 'annual_contract_value', 'product_tier',
       'sales_segment', 'acquisition_channel', 'contract_start_date',
       'contract_end_date', 'renewed_flag', 'discount_pct',
       'initial_onboarding_score', 'is_churned', 'Q3_Status', 'channel',
       'priority', 'issue_category', 'ticket_text', 'avg_sentiment',
       'avg_first_response_hours', 'avg_resolution_hours', 'avg_resolved',
       'ticket_count', 'hierarchical_cluster', 'lost_revenue'],
      dtype='object')


In [ ]:
target = ["lost_revenue"]
features = []

In [25]:
joined.isna().sum()

customer_id                    0
company_name                   0
country                        0
region                         0
is_eu                          0
industry                       0
company_size_bucket            0
annual_contract_value          0
product_tier                   0
sales_segment                  0
acquisition_channel            0
contract_start_date            0
contract_end_date           3276
renewed_flag                   0
discount_pct                   0
initial_onboarding_score       0
is_churned                     0
Q3_Status                    579
channel                      579
priority                     579
issue_category               579
ticket_text                  579
avg_sentiment                579
avg_first_response_hours     579
avg_resolution_hours         579
avg_resolved                 579
ticket_count                 579
hierarchical_cluster           0
lost_revenue                   0
dtype: int64